In [25]:

import csv
import tensorflow as tf
import numpy as np
import urllib


In [26]:
# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [27]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
urllib.request.urlretrieve(url, 'sunspots.csv')

('sunspots.csv', <http.client.HTTPMessage at 0x19eabec50d0>)

In [28]:
timestep = []
sunspots = []
with open('Sunspots.csv') as csvfile:
    reader=csv.reader(csvfile)
    next(reader)
    for line in reader:
        timestep.append(int(line[0]))
        sunspots.append(float(line[2]))

In [29]:

series = np.array(sunspots)


In [30]:

# DO NOT CHANGE THIS CODE
# This is the normalization function
min = np.min(series)
max = np.max(series)
series -= min
series /= max
time = np.array(time_step)

In [31]:
split_time = 3000

In [32]:

time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]


In [33]:
window_size = 30
batch_size = 32
shuffle_buffer_size = 1000

In [34]:
train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)

In [35]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=32, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1)
])

In [36]:
optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set, epochs=100)

Epoch 1/100
93/93 [==============================] - 1s 14ms/step - loss: 0.0345 - mae: 0.2017
Epoch 2/100
93/93 [==============================] - 1s 14ms/step - loss: 0.0335 - mae: 0.1969
Epoch 3/100
93/93 [==============================] - 1s 14ms/step - loss: 0.0325 - mae: 0.1921
Epoch 4/100
93/93 [==============================] - 1s 14ms/step - loss: 0.0314 - mae: 0.1875
Epoch 5/100
93/93 [==============================] - 1s 14ms/step - loss: 0.0304 - mae: 0.1832
Epoch 6/100
93/93 [==============================] - 1s 14ms/step - loss: 0.0295 - mae: 0.1791
Epoch 7/100
93/93 [==============================] - 1s 15ms/step - loss: 0.0286 - mae: 0.1754
Epoch 8/100
93/93 [==============================] - 1s 14ms/step - loss: 0.0278 - mae: 0.1719
Epoch 9/100
93/93 [==============================] - 1s 15ms/step - loss: 0.0269 - mae: 0.1688
Epoch 10/100
93/93 [==============================] - 1s 14ms/step - loss: 0.0262 - mae: 0.1659
Epoch 11/100
93/93 [=============================

93/93 [==============================] - 1s 15ms/step - loss: 0.0134 - mae: 0.1287
Epoch 87/100
93/93 [==============================] - 1s 15ms/step - loss: 0.0134 - mae: 0.1287
Epoch 88/100
93/93 [==============================] - 1s 15ms/step - loss: 0.0134 - mae: 0.1287
Epoch 89/100
93/93 [==============================] - 1s 15ms/step - loss: 0.0133 - mae: 0.1286
Epoch 90/100
93/93 [==============================] - 1s 15ms/step - loss: 0.0133 - mae: 0.1286
Epoch 91/100
93/93 [==============================] - 1s 15ms/step - loss: 0.0133 - mae: 0.1286
Epoch 92/100
93/93 [==============================] - 1s 15ms/step - loss: 0.0133 - mae: 0.1286
Epoch 93/100
93/93 [==============================] - 1s 15ms/step - loss: 0.0133 - mae: 0.1285
Epoch 94/100
93/93 [==============================] - 1s 14ms/step - loss: 0.0133 - mae: 0.1285
Epoch 95/100
93/93 [==============================] - 1s 14ms/step - loss: 0.0132 - mae: 0.1285
Epoch 96/100
93/93 [==============================] -

In [37]:
import math 
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

window_size = 30
rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()
test_val = 100 * result
score = math.ceil(17 - test_val)
if score > 5:
    score = 5
print(result)
print(score)

0.113982946
5
